In [6]:
# =========================
# 1. Import Libraries
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pymc as pm
# import theano.tensor as tt
# =========================
# 2. Load Data
# =========================
oil_df = pd.read_csv("../data/raw/BrentOilPrices.csv", parse_dates=["Date"])
events_df = pd.read_csv("../data/raw/oil_market_events.csv", parse_dates=["date"])

oil_df = oil_df.sort_values("Date").dropna(subset=["Price"])
price_data = oil_df["Price"].values
n_data = len(price_data)

C:\Users\user\AppData\Local\Temp\ipykernel_15308\4227307412.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  oil_df = pd.read_csv("../data/raw/BrentOilPrices.csv", parse_dates=["Date"])


In [9]:
# =========================
# 3. Quick Plot of Price
# =========================
import matplotlib.pyplot as plt
plt.figure(figsize=(15,6))
plt.plot(oil_df["Date"], oil_df["Price"], label="Brent Oil Price", color="blue")
plt.title("Brent Oil Prices (1987–2022)")
plt.xlabel("Date")
plt.ylabel("Price (USD per Barrel)")
plt.legend()
plt.show()

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "c:\Users\user\Desktop\bayesian-change-point-oil\venv\Scripts\python.exe"
  * The NumPy version is: "1.22.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: numpy._core.multiarray failed to import

In [ ]:
# =========================
# 4. Bayesian Change Point Model
# =========================
with pm.Model() as model:
    # Change point location prior
    tau = pm.DiscreteUniform("tau", lower=0, upper=n_data-1)

    # Mean prices before & after
    mu_1 = pm.Normal("mu_1", mu=np.mean(price_data), sd=np.std(price_data))
    mu_2 = pm.Normal("mu_2", mu=np.mean(price_data), sd=np.std(price_data))

    # Shared sigma
    sigma = pm.HalfNormal("sigma", sd=10)

    # Switch mean based on tau
    mu = tt.switch(tau >= np.arange(n_data), mu_1, mu_2)

    # Likelihood
    likelihood = pm.Normal("likelihood", mu=mu, sd=sigma, observed=price_data)

    # Sample posterior
    trace = pm.sample(2000, tune=1000, target_accept=0.95, cores=2, random_seed=42)


In [ ]:
# =========================
# 5. Interpret Results
# =========================
tau_median = int(np.median(trace["tau"]))
change_date = oil_df.iloc[tau_median]["Date"]

mean_before = np.mean(price_data[:tau_median])
mean_after = np.mean(price_data[tau_median:])
impact_pct = ((mean_after - mean_before) / mean_before) * 100

print(f"📅 Most probable change point date: {change_date.date()}")
print(f"Before change: ${mean_before:.2f}")
print(f"After change: ${mean_after:.2f}")
print(f"Impact: {impact_pct:.2f}% change in mean price")

In [ ]:
# =========================
# 6. Match with Known Events
# =========================
events_df["date_diff"] = abs((events_df["date"] - change_date).dt.days)
closest_event = events_df.loc[events_df["date_diff"].idxmin()]

print("\nClosest known event:")
print(f"Event: {closest_event['event']}")
print(f"Type: {closest_event['type']}")
print(f"Description: {closest_event['description']}")

In [ ]:
# =========================
# 7. Plot with Change Point
# =========================
plt.figure(figsize=(15,6))
plt.plot(oil_df["Date"], oil_df["Price"], label="Brent Oil Price", color="blue")
plt.axvline(change_date, color="red", linestyle="--", label="Detected Change Point")
plt.title("Brent Oil Price with Bayesian Change Point")
plt.xlabel("Date")
plt.ylabel("Price (USD per Barrel)")
plt.legend()
plt.show()